In [1]:
# declaraciones de librerías
using DataStructures

# Grafos

Antes de explicar y desarrollar algoritmos específicos, debemos comentar un poco las diferentes estructuras y algunas decisiones de implementación realizadas, así como implementar un par de generadores de grafos.

## Limitaciones

* Los códigos que vamos a desarrollar son explicativos, prima la claridad y la exposición de algunas limitaciones sobre la versatilidad.
* Existen librerías para grafos en Julia (https://github.com/JuliaGraphs/LightGraphs.jl y https://github.com/JuliaArchive/Graphs.jl entre otras).
* Reimplementaremos muchas cosas. Es el precio a pagar por intentar explicar algoritmos y no hacer una librería de grafos.
* Trataremos un grafo no dirigido como un grafo dirigido con los arcos. Esto es una solución válida para los problemas y algoritmos que veremos aquí pero muy discutible para otros problemas (como Spanning Tree)

## Tipos de grafos

Veremos tres tipos principales de grafos

* Grafos densos. En un grafo denso (ratio grande de arcos / aristas) es conveniente guardar la informaión de los arcos a través de una matriz.
* Grafos "sparse" tipo (I). Son los grafos en que analizaremos arcos pensando en su vértice origen. De forma similar podríamos discutir aquellos grafos en que los arcos se analizan pensando en su vértice destino.
* Grafos "sparse" tipo (II). Queremos referenciarnos a un arco tanto por su origen como por su destino.

## Algoritmos analizados

Veremos los siguientes problemas y algoritmos:

* Caminos extremos desde un vértice origen
    - Dijkstra
    - Bellman-Ford
* Caminos extremos entre toda pareja de vértices
    - Floyd-Warshall
* Problemas de flujos máximos
    - Ford-Fulkerson
    - Edmonds-Karp
    - Dinic
    - Goldberg-Tarjan (Push-relabel)
* Problemas de flujos máximos a costo mínimo
    - Minimum mean cycle-cancelling
    - Successive shortest path
    - Capacity Scaling
    - Orlin

## Otros

Por primera vez vamos a llamar otro archivo desde uno de nuestros libros. El código empieza a ser demasiado grande como para usar una hoja de Jupyter

## Referencias

Hay mucha literatura sobre el tema:

* Posiblemente el libro más completo es "Network Flows: Theory, Algorithms, and Applications" de Ahuja, Magnanti y Orlin.
* Alternativamente y si no se ne esita trabajar con costes "Introduction to algorithms" de Cormen, Leiserson, Rivest y Stein es bastante más claro, menos enciclopédico y, a mi entender, contiene más información que hacen de él un muy buen libro de referencia.
* Finalmente, yo uso mucho "Combinatorial Optimization. Theory and Algorithms" de Korte y Vygen. El material de estas clases se basa principalmente en este tercer libro.

# Grafo

Como mínimo, un grafo queda definido por su número de vértices, y por la existencia de un arco entre pareja de vértices. 

Nota: esta definición es incompleta y directamente falsa, pero será la que usemos en el tema. Estamos descartando entre otros, grafos en los que hay arcos que unen varios vértices (más de dos) y grafos en que hay más de un arco o arista uniendo una pareja de vértices

In [81]:
type gSimple
    vertices::Int64
    arcos::Array{Bool,2} #arco[i,j]=true indica que existe un arco entre i y j 
end

In [82]:
function generarGrafoSimple(n::Int64,sp::Float64)
    a=Array{Bool}(n,n)
    for i in 1:n
        for j in 1:n
            if rand()<sp
                a[i,j]=true
            else
                a[i,j]=false
            end
        end
    end
    return gSimple(n,a) 
end


g=generarGrafoSimple(5,0.2)
println("estoy En: ",g)

estoy En: gSimple(5,Bool[false false false false false; true true false false false; false false false false false; false false true false false; false false false false false])


Este tipo de grafos ya nos nos permitirá probar dos ideas generales en grafos. La primera es la exploración primero en profundidad (depth-first) y la segunda es la exploración primero en anchura (breadth-first). 

Usaremos ambos métodos para testear conectividad.

In [83]:
function DFS_visit(G::gSimple,u::Int64,color::BitArray{1},d::Array{Int64,1},π::Array{Int64,1},depth::Int64)
    d[u]=depth
    color[u]=true
    for v in 1:G.vertices
        if color[v]==false && G.arcos[u,v]==true
            π[v]=u
            DFS_visit(G,v,color,d,π,depth+1)
        end
    end
end

function conectividadDepthFirst(G::gSimple,origen::Int64)
    color=falses(G.vertices)
    π=Array(Int64,G.vertices)
    d=Array(Int64,G.vertices)
    fill!(d,-1)
    color[origen]=true
    d[origen]=0
    for i in 1:G.vertices
        if color[i]==false && G.arcos[origen,i]==true
            DFS_visit(G, i,color,d,π,1)
        end
    end
    return color
end

function conectividadBreadthFirst(G::gSimple,origen::Int64)
    ∞=1000000
    #arrays para construir evaluación
    color=falses(G.vertices)
    println("color: ",color)
    d=Array(Int64,G.vertices)
    fill!(d,∞)
    π=Array(Int64,G.vertices)
    fill!(d,-1)
    #inicializamos para el vertice
    color[origen]=true
    d[origen]=0
    Q = Deque{Int64}()
    push!(Q,origen)
    while isempty(Q)==false
        u=pop!(Q)
        for i in 1:G.vertices
            #println("\t",i,"\t",u,"\t",G.arcos[u,i])
            if color[i]==false && G.arcos[u,i]==true
                color[i]=true
                d[i]=d[u]+1
                π[i]=u
                push!(Q,i)
            end
        end
    end
    return color
end

srand(0)
G = generarGrafoSimple(10,0.15)
conectados = conectividadBreadthFirst(G,1)
println("conectados a ",1," : ",countnz(conectados))
#println(conectados)
conectadosBis = conectividadDepthFirst(G,1)
println("conectados a ",1," : ",countnz(conectadosBis))
#println(conectadosBis)



color: Bool[false,false,false,false,false,false,false,false,false,false]
conectados a 1 : 8
conectados a 1 : 8


## Grafo disperso

Una alternativa sería tener en cuenta grafos dispersos (aquellos que tiene muchos menos arcos de lo que podrían tener)

En tal caso la estructura sería un tanto diferente. Empezaremos por una versión simple, como en el caso anterior

In [1]:
type gSimpleSparse
    vertices::Int64
    aristas::Int64
    pOrigen::Array{Int64,1}
    destino::Array{Int64,1}
end

In [2]:
function generarGrafoSimpleSparse(n::Int64,sp::Float64)
    a=1
    pOrigen=Int64[]
    destino=Int64[]
    for i in 1:n
        push!(pOrigen,a)
        for j in 1:n
            if rand()<sp
                push!(destino,j)
                a += 1
            end
        end
    end
    push!(pOrigen,a)
    return(gSimpleSparse(n,a-1,pOrigen,destino))
end


g=generarGrafoSimpleSparse(5,0.2)
println("estoy En: ",g)

estoy En: gSimpleSparse(5,6,[1,3,5,5,6,7],[4,5,4,5,3,1])


In [8]:
function DFS_visitSparse(G::gSimpleSparse,u::Int64,color::BitArray{1},d::Array{Int64,1},π::Array{Int64,1},depth::Int64)
    d[u]=depth
    color[u]=true
    for c in G.pOrigen[u]:G.pOrigen[u+1]-1
        v=G.destino[c]
        if color[v]==false 
            π[v]=u
            DFS_visitSparse(G,v,color,d,π,depth+1)
        end
    end
end

function conectividadDepthFirstSparse(G::gSimpleSparse,origen::Int64)
    color=falses(G.vertices)
    π=Array(Int64,G.vertices)
    d=Array(Int64,G.vertices)
    fill!(d,-1)
    color[origen]=true
    d[origen]=0
    for c in G.pOrigen[origen]:G.pOrigen[origen+1]-1
        i=G.destino[c]
        if color[i]==false
            DFS_visitSparse(G, i,color,d,π,1)
        end
    end
    return color
end

function conectividadBreadthFirstSparse(G::gSimpleSparse,origen::Int64)
    ∞=1000000
    #arrays para construir evaluación
    color=falses(G.vertices)
    d=Array(Int64,G.vertices)
    fill!(d,∞)
    π=Array(Int64,G.vertices)
    fill!(d,-1)
    #inicializamos para el vertice
    color[origen]=true
    d[origen]=0
    Q = Deque{Int64}()
    push!(Q,origen)
    while isempty(Q)==false
        u=pop!(Q)
        for c in G.pOrigen[u]:G.pOrigen[u+1]-1
            i=G.destino[c]
            #println("\t",i,"\t",u,"\t",G.arcos[u,i])
            if color[i]==false
                color[i]=true
                d[i]=d[u]+1
                π[i]=u
                push!(Q,i)
            end
        end
    end
    return color
end

srand(0)
G = generarGrafoSimpleSparse(10,0.15)
conectados = conectividadBreadthFirstSparse(G,1)
println("conectados a ",1," : ",countnz(conectados))
println(conectados)

conectadosBis = conectividadDepthFirstSparse(G,1)
println("conectados a ",1," : ",countnz(conectadosBis))
println(conectadosBis)


conectados a 1 : 8
Bool

[true,true,true,true,true,false,true,true,false,true]
conectados a 1 : 8
Bool[true,true,true,true,true,false,true,true,false,true]


### Ejercicios

¿Podría implementarse depth first sin recursividad? ¿Cómo?

## Estructuras

En varias ocasiones vamos a necesitar estructuras de datos con mayor información o información expresada de otras maneras. 

Primero, debemos pensar si el grafo va a ser completo o sparse. La estructura para grafos densos siempre se basará en matrices (puede que triangulares superiores). 

Por ejemplo para shortest paths:

In [6]:
type gPaths
    vertices::Int64
    arcos::Array{Int64,2} #arco[i,j] indica la distancia entre i y j 
end

type arcoPaths
    origen::Int64
    destino::Int64
    longitud::Int64
end

type gPathsSparse
    nVertices::Int64
    nArcos::Int64
    pOrigen::Array{Int64,1}
    arcos::Array{arcoPaths,1} #arco[i,j] indica la distancia entre i y j 
end

function generarGrafoDense(n::Int64,maxValue::Int64)
    a=Array{Int64}(n,n)
    rand!(a,1:maxValue)
    for i in 1:n
        a[i,i]=0
    end
    return gPaths(n,a) 
end

function generarGrafoSparse(n::Int64,sp::Float64,maxValue::Int64)
    a=arcoPaths[]
    pOrigen=Int64[]
    nArcos=1
    for i in 1:n
        push!(pOrigen,nArcos)
        for j in 1:n
            if i!=j
                if rand()<sp
                    push!(a,arcoPaths(i,j,rand(1:maxValue)))
                    nArcos += 1
                end
            end
        end
    end
    return gPathsSparse(n,nArcos-1,pOrigen,a) 
end


generarGrafoSparse (generic function with 1 method)

In [7]:
println(generarGrafoDense(5,1000))
println(generarGrafoSparse(5,0.25,1000))

gPaths(5,[0 843 272 994 444; 481 0 659 701 952; 24 858 0 280 756; 759 520 681 0 131; 904 711 48 921 0])
gPathsSparse(5,6,[1,3,4,5,5],arcoPaths[arcoPaths(1,2,80),arcoPaths(1,3,561),arcoPaths(2,3,763),arcoPaths(3,1,62),arcoPaths(5,1,917),arcoPaths(5,3,260)])
